In [1]:
%cd .. 

/Users/nguyenhuyhai/20221/lab_blockchain/orchai_validator


In [2]:
from labeling.ETLProcessor import ETLProcessor
from labeling.tools import get_spark, query
from omegaconf import OmegaConf

In [3]:
config = OmegaConf.load("config/local.yaml")
spark = get_spark('lib/postgresql-42.5.0.jar')

df = query(spark, **config.src)
df.printSchema()

22/12/09 14:28:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Successfully queried data from database
root
 |-- block_height: integer (nullable = true)
 |-- operator_address: string (nullable = true)
 |-- jailed: boolean (nullable = true)
 |-- status: string (nullable = true)
 |-- tokens: decimal(38,0) (nullable = true)
 |-- commission_rate: float (nullable = true)
 |-- delegator_shares: decimal(38,0) (nullable = true)
 |-- self_bonded: decimal(38,0) (nullable = true)
 |-- propose: boolean (nullable = true)
 |-- vote: boolean (nullable = true)



In [4]:
accept_rate = 0.1
concentration_level = 0.9
vote_score = 2
propose_score = 6
A = 9
B = 4
C = 2
D = 4
"""
suitable parameters are:
vote_proposed_win_size = 14400 (blocks) 
combine_win_size = 150 (blocks)
label_win_size = 100800 (blocks)
Approximate each block ~~ 6seconds
"""
vote_proposed_win_size = 500 #This is just test, favourable is
combine_win_size = 20 
label_win_size = 60


df = ETLProcessor.data_scoring(df, accept_rate, concentration_level, vote_score, propose_score, A, B, C, D,vote_proposed_win_size, combine_win_size, label_win_size)


------------------------------------------------
Successfully converted voting_power_score column
------------------------------------------------
------------------------------------------------
Successfully converted commission_score column
------------------------------------------------
------------------------------------------------
Successfully converted self_bonded_score column
------------------------------------------------
------------------------------------------------
Successfully converted vote_propose_score column
------------------------------------------------
------------------------------------------------
Sucessfully converted final_score
------------------------------------------------
------------------------------------------------
Sucessfully combine data
------------------------------------------------
------------------------------------------------
Sucessfully shifting data
------------------------------------------------


In [5]:
# df.count()

In [7]:
df.show(1000)

22/12/09 14:30:33 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (10.130.235.115 executor driver): TaskKilled (Stage cancelled)
22/12/09 14:30:48 WARN TaskSetManager: Lost task 0.0 in stage 1.0 (TID 1) (10.130.235.115 executor driver): TaskKilled (Stage cancelled)


+---------+--------------------+--------------------+------------------+-------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------------------+------------------+
|new_block|    operator_address|     comission_score|             score| voting_power_score|tokens_proportion|     commission_rate|         self_bonded|   self_bonded_score|    delegator_shares|           tokens|block_height|vote_propose_score|             label|
+---------+--------------------+--------------------+------------------+-------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+------------------+------------------+
|        0|oraivaloper1n4xkr...|  0.6999999918230734|13.595477070062028| 0.9164430999999997|     0.0026530000|0.030000000817692673|45579058200000000...|                 0.0|21992059860000000...|  2199205986.0

In [ ]:
df.printSchema()

In [ ]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()


In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType


In [ ]:
df_test = spark.createDataFrame(
[(1,2,"a"),
(2,2,"a"),
(3,3,"a"),
(4,2,"a"),
(5,3,"a"),
(6,3,"a"),
(7,3,"a"),
(8,3,"a"),
(9,3,"a"),
(1,3,"b"),
(2,3,"b"),
(3,3,"b"),
(4,3,"b"),
(5,3,"b"),
(6,3,"b"),
(7,3,"b"),
(8,3,"b"),
(9,3,"b"),
],  
["block_height", "score", "operator_address"])

In [ ]:
def shifting_data(df: DataFrame, label_win_size: int, combine_win_size: int):
        size = int(label_win_size/combine_win_size) # As we'll shift data after combining the data
        window = Window.partitionBy("operator_address").orderBy("block_height").rangeBetween(0, size)
        #window is used for shifting "size" blocks and calculate the mean
        lag_window = Window.partitionBy("operator_address").orderBy("block_height")
        #lag_window is used for getting the null blocks - the last blocks that do not change

        df = df.withColumn(
            "label", F.when(
                        F.lag("score", -size).over(lag_window).isNotNull(), F.sum("score").over(window)
        ))\
        .orderBy("operator_address", "block_height")
        # df = df.drop("score")
        df = df.na.drop()
        return df

In [ ]:
df_test = shifting_data(df_test, 6,3 )

In [ ]:
df_test.show()

In [ ]:
int(6/3)